# import libraries

In [528]:
import numpy as np
import pandas as pd
import re
from collections import Counter
from xgboost import XGBClassifier, XGBRegressor, plot_importance
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn import linear_model
from numpy import sort

In [529]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [530]:
def to_total_months(s):
    result = re.match(r'(\d+) years and (\d+) months?', s)
    if result:
        years = int(result.group(1))
        months = int(result.group(2))
        total_months = years * 12 + months
        return total_months
    else:
        return int(s.split()[0])

In [531]:
def preprocess_data(df):
    policy_ids = df['policy_id']
    df = df.drop('policy_id', axis=1)
    df = df.drop(df.columns[:1], axis = 1)

    df['age_of_car'] = df['age_of_car'].apply(to_total_months)

    df = pd.get_dummies(df, columns=['area_cluster', 'make', 'segment', 'model', 'fuel_type', \
                                        'engine_type', 'rear_brakes_type', 'transmission_type',\
                                        'steering_type'])
    df = df.replace({True: 1, False: 0})

    df[['torque_Nm', 'torque_rpm']] = df['max_torque'].str.split('Nm@', expand=True)
    df[['power_Nm', 'power_rpm']] = df['max_power'].str.split('bhp@', expand=True)

    df['torque_rpm'] = df['torque_rpm'].str.rstrip('rpm')
    df['power_rpm'] = df['power_rpm'].str.rstrip('rpm')   

    df['torque_Nm'] = pd.to_numeric(df['torque_Nm'])
    df['torque_rpm'] = pd.to_numeric(df['torque_rpm'])
    df['power_Nm'] = pd.to_numeric(df['power_Nm'])
    df['power_rpm'] = pd.to_numeric(df['power_rpm'])

    for col in df.columns:
        if re.search(r'is_', col):
            df[col] = df[col].replace({'Yes': 1, 'No': 0})
        else:
            continue

    df['volume'] = df.apply(lambda row:np.cbrt(row['length'] * row['width'] * row['height']), axis=1)
    
    df = df.drop(columns=['max_torque', 'max_power', 'length', 'width', 'height'], axis=1)
    return policy_ids, df 

In [532]:
policy_ids_train, df_train = preprocess_data(train_df)
policy_ids_test, df_test = preprocess_data(test_df)

/var/folders/tn/m3w4cr2j7csb0n3r_j673p3w0000gn/T/ipykernel_88158/3736461425.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})
/var/folders/tn/m3w4cr2j7csb0n3r_j673p3w0000gn/T/ipykernel_88158/3736461425.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'Yes': 1, 'No': 0})
/var/folders/tn/m3w4cr2j7csb0n3r_j673p3w0000gn/T/ipykernel_88158/3736461425.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To reta

In [547]:
# xgboost
X_train = df_train.drop('age_of_policyholder', axis=1).values
X_test = df_test.drop('age_of_policyholder', axis=1).values
y_train = df_train['age_of_policyholder'].values
y_test = df_test['age_of_policyholder'].values


# fit model
# 开始训练
model_regression = XGBRegressor()
model_regression.fit(X_train, y_train)


# make predictions
y_pred = model_regression.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 93.36543270424967


In [548]:
cv_params = {'reg_alpha':[0.1, 0.2],
             'reg_lambda': [0.5, 0.7, 1]}

other_params = {'subsample':0.8,
                'colsample_bytree':0.8,
                'min_child_weight':3,
                'seed':0,
                'n_estimators':150,
                'gamma':0.125,
                'learning_rate':0.05, 
                'max_depth': 4}

model_regression = XGBRegressor(**other_params)
opti = GridSearchCV(estimator=model_regression, param_grid=cv_params, scoring='neg_mean_squared_error', n_jobs=-1, verbose=3)
opti.fit(X_train, y_train)
res = opti.cv_results_
print(f'每轮最佳得分:{res}')
print(f'参数的最佳取值：{opti.best_params_}')
print(f'最佳模型得分:{opti.best_score_}')
other_params.update(opti.best_params_)
model_regression = XGBRegressor(**other_params)
model_regression.fit(X_train,y_train)
y_pred = model_regression.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(mse)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ...reg_alpha=0.1, reg_lambda=0.5;, score=-91.180 total time=   0.9s
[CV 2/5] END ...reg_alpha=0.1, reg_lambda=0.5;, score=-93.086 total time=   0.9s
[CV 3/5] END ...reg_alpha=0.1, reg_lambda=0.5;, score=-90.051 total time=   0.9s
[CV 4/5] END ...reg_alpha=0.1, reg_lambda=0.5;, score=-91.057 total time=   0.9s
[CV 5/5] END ...reg_alpha=0.1, reg_lambda=0.5;, score=-90.903 total time=   0.9s
[CV 1/5] END ...reg_alpha=0.1, reg_lambda=0.7;, score=-91.204 total time=   0.9s
[CV 2/5] END ...reg_alpha=0.1, reg_lambda=0.7;, score=-93.048 total time=   0.9s
[CV 3/5] END ...reg_alpha=0.1, reg_lambda=0.7;, score=-90.076 total time=   0.9s
[CV 5/5] END ...reg_alpha=0.1, reg_lambda=0.7;, score=-90.895 total time=   0.8s
[CV 4/5] END ...reg_alpha=0.1, reg_lambda=0.7;, score=-91.049 total time=   0.9s
[CV 2/5] END .....reg_alpha=0.1, reg_lambda=1;, score=-93.070 total time=   0.8s
[CV 3/5] END .....reg_alpha=0.1, reg_lambda=1;, s

In [549]:
thresholds = sort(model_regression.feature_importances_)
thresholds = thresholds[ thresholds != 0]
min_mse = mse
output_pred = y_pred
tmp_x_train = X_train
tmp_x_test = X_test
for thresh in thresholds:
	# select features using threshold
	selection = SelectFromModel(model_regression, threshold=thresh, prefit=True)
	select_X_train = selection.transform(X_train)
	# train model
	selection_model = XGBRegressor(**other_params)
	selection_model.fit(select_X_train, y_train)
	# eval model
	select_X_test = selection.transform(X_test)
	predictions = selection_model.predict(select_X_test)
	mse = mean_squared_error(y_test, predictions)
	if mse < min_mse:
		print("Mean Squared Error:", mse, f'n={select_X_train.shape[1]}', f'thresh={thresh}')
		min_mse = mse
		output_pred = predictions
		tmp_x_train = select_X_train
		tmp_x_test = select_X_test
print(f'final minimum mse: {min_mse}')

final minimum mse: 90.80894402861882


In [550]:
output_pred

array([38.647633, 38.66945 , 36.771816, ..., 35.211792, 34.752808,
       36.97485 ], dtype=float32)

In [551]:
output_part1 = pd.DataFrame({'policy_id': policy_ids_test, 'age_of_policyholder': output_pred})
output_part1.to_csv('z5364634.PART1.output.csv', index=False)

# classifier

In [553]:
X_train = df_train.drop('is_claim', axis=1).values
X_test = df_test.drop('is_claim', axis=1).values
y_train = df_train['is_claim'].values
y_test = df_test['is_claim'].values

In [649]:
model_classification = XGBClassifier(scale_pos_weight=6)
model_classification.fit(X_train, y_train)

# make predictions
y_pred = model_classification.predict(X_test)
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1 Score :", macro_f1)

Macro F1 Score : 0.5439759219286778


In [749]:
other_params = {
                'colsample_bytree':0.7,
                'max_depth':6,
                'min_child_weight':4,
                'reg_alpha':0.4,
                'n_estimators':200,
                'scale_pos_weight':6,
                }
model_classification = XGBClassifier(**other_params)
model_classification.fit(X_train,y_train)
y_pred = model_classification.predict(X_test)
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1 Score :", macro_f1)

Macro F1 Score : 0.5432493243068808


In [658]:
cv_params = {
            'reg_lambda' :[0, 0.1, 0.5, 1]
            }

other_params = {
                # 'colsample_bytree':0.8,
                # 'max_depth':6,
                # 'min_child_weight':4,
                # 'reg_alpha':0.4,
                'n_estimators':125,
                # 'learning_rate':0.01,
                'scale_pos_weight':6
                }

model_classification = XGBClassifier(**other_params)
opti = GridSearchCV(estimator=model_classification, param_grid=cv_params, scoring='f1_macro', n_jobs=-1, verbose=3)
opti.fit(X_train, y_train)
res = opti.cv_results_
print(f'每轮最佳得分:{res}')
print(f'参数的最佳取值：{opti.best_params_}')
print(f'最佳模型得分:{opti.best_score_}')
other_params.update(opti.best_params_)
model_classification = XGBClassifier(**other_params)
model_classification.fit(X_train,y_train)
y_pred = model_classification.predict(X_test)
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1 Score :", macro_f1)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ....................reg_lambda=0.1;, score=0.529 total time=   1.1s
[CV 3/5] END ....................reg_lambda=0.1;, score=0.522 total time=   1.0s
[CV 2/5] END ......................reg_lambda=0;, score=0.511 total time=   1.2s
[CV 3/5] END ......................reg_lambda=0;, score=0.518 total time=   1.2s
[CV 1/5] END ......................reg_lambda=0;, score=0.529 total time=   1.3s
[CV 5/5] END ......................reg_lambda=0;, score=0.523 total time=   1.2s
[CV 2/5] END ....................reg_lambda=0.1;, score=0.515 total time=   1.1s
[CV 4/5] END ......................reg_lambda=0;, score=0.519 total time=   1.3s
[CV 4/5] END ....................reg_lambda=0.1;, score=0.523 total time=   0.8s
[CV 2/5] END ....................reg_lambda=0.5;, score=0.519 total time=   0.9s
[CV 5/5] END ....................reg_lambda=0.1;, score=0.527 total time=   0.9s
[CV 5/5] END ....................reg_lambda=0.5;,

In [750]:
thresholds = sort(model_classification.feature_importances_)
thresholds = thresholds[thresholds != 0]
max_f1 = macro_f1
tmp_x_train = X_train
tmp_x_test = X_test
output_preds = None
for thresh in thresholds:
	# select features using threshold
	selection = SelectFromModel(model_classification, threshold=thresh, prefit=True)
	select_X_train = selection.transform(X_train)
	# train model
	selection_model = XGBClassifier(**other_params)
	selection_model.fit(select_X_train, y_train)
	# eval model
	select_X_test = selection.transform(X_test)
	predictions = selection_model.predict(select_X_test)
	macro_f1 = f1_score(y_test, predictions, average='macro')
	if macro_f1 > max_f1:
		print("Macro f1 score:", macro_f1, f'n={select_X_train.shape[1]}')
		max_f1 = macro_f1
		output_preds = predictions
		tmp_x_train = select_X_train
		tmp_x_test = select_X_test
print(f'final maximum: {max_f1}')

Macro f1 score: 0.5434666666666667 n=43
final maximum: 0.5434666666666667


In [751]:
print(thresholds)

[0.00034973 0.00149463 0.00338924 0.00369296 0.00422056 0.00485526
 0.00521619 0.00549205 0.00620633 0.00626181 0.00659561 0.00675782
 0.00677731 0.0067961  0.0069307  0.00714848 0.00739473 0.00741154
 0.00743585 0.00782781 0.00793017 0.00830402 0.00834823 0.00845489
 0.00857781 0.00862961 0.00911002 0.00935323 0.00937375 0.00947312
 0.00952946 0.00988551 0.00996889 0.01009746 0.01024465 0.01033334
 0.01035878 0.01050217 0.01059569 0.0105965  0.01067591 0.01068363
 0.01076066 0.01088573 0.01100071 0.01102172 0.01118556 0.01133658
 0.01139579 0.01148286 0.01154514 0.01174338 0.01189518 0.01193083
 0.01213954 0.01232637 0.01240958 0.01242332 0.0125062  0.01250928
 0.01263651 0.01284874 0.01295598 0.01319087 0.01325432 0.01328575
 0.01330655 0.0133824  0.01338926 0.01356688 0.01356698 0.0137939
 0.01408    0.01428014 0.0143791  0.01462148 0.01561455 0.01574687
 0.015922   0.01677261 0.01889106 0.020113   0.02013531 0.02040065
 0.02965249 0.03272917 0.04370343]


In [542]:
output_preds

array([0, 0, 0, ..., 0, 0, 1])

In [543]:
output_part2 = pd.DataFrame({'policy_id': policy_ids_test, 'age_of_policyholder': output_preds})
output_part2.to_csv('z5364634.PART2.output.csv', index=False)

In [1]:
import pandas as pd

# 创建示例DataFrame
data = {
    'Date': ['2024-04-01', '2024-04-01', '2024-04-02', '2024-04-02', '2024-04-03'],
    'Product Category': ['A', 'B', 'A', 'B', 'A'],
    'Sales Quantity': [100, 150, 120, 130, 110],
    'Sales Amount': [1000, 1500, 1200, 1300, 1100]
}

df = pd.DataFrame(data)
print(df)

         Date Product Category  Sales Quantity  Sales Amount
0  2024-04-01                A             100          1000
1  2024-04-01                B             150          1500
2  2024-04-02                A             120          1200
3  2024-04-02                B             130          1300
4  2024-04-03                A             110          1100


/var/folders/tn/m3w4cr2j7csb0n3r_j673p3w0000gn/T/ipykernel_9240/733549182.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# 创建数据透视表
pivot_table = pd.pivot_table(df, index='Date', columns='Product Category', 
                              values=['Sales Quantity', 'Sales Amount'], 
                              )

print(pivot_table)

                 Sales Amount         Sales Quantity       
Product Category            A       B              A      B
Date                                                       
2024-04-01             1000.0  1500.0          100.0  150.0
2024-04-02             1200.0  1300.0          120.0  130.0
2024-04-03             1100.0     NaN          110.0    NaN


In [4]:
import pandas as pd

data = {'timestamp': ['2019-21-01:11:59:12 AM', '2020-12-05:08:30:45 PM']}
df = pd.DataFrame(data)

# 将字符串转换为 datetime 对象
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%d-%m:%I:%M:%S %p')

# 重新格式化时间，以年-月-日-24小时制的形式
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d-%H:%M:%S')


print(df)

             timestamp
0  2019-01-21-11:59:12
1  2020-05-12-20:30:45
